In [ ]:
import os
import openai
import anthropic
import together
import subprocess
import re

# open ai API key
openai_api_key=''

# anthropic API key
claude_api_key = ''

# together AI API key
together_api_key = ''

In [4]:
# read the prompt from the prompt text file titled "facilitysizing.txt"
with open("../prompts/facsize-1.txt", "r") as f:
    prompt = f.read()

prompt = str(prompt)

In [ ]:
print(prompt)

In [6]:
claude_client = anthropic.Anthropic(api_key = claude_api_key)
openai_client = openai.OpenAI(api_key = openai_api_key)
togetherai_client = together.Together(api_key=together_api_key)

## GPT 4o

In [7]:
completion = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [ ]:
print(response)

In [23]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

def simulate_demand(mu, Sigma, num_samples):
    return np.random.multivariate_normal(mu, Sigma, num_samples)

def estimate_p_stockout(capacities, demands):
    stockouts = np.any(demands > capacities, axis=1)
    return np.mean(stockouts)

def objective(x, demands, costs):
    return np.sum(x * costs)

def constraint(x, demands, epsilon):
    p_stockout = estimate_p_stockout(x, demands)
    return epsilon - p_stockout

def solve_optimization(mu, Sigma, costs, epsilon, budget):
    num_samples = budget // 10  # A heuristic choice for sample size
    demands = simulate_demand(mu, Sigma, num_samples)

    initial_guess = np.array([150, 300, 400])  # Original guess
    bounds = [(0, None) for _ in initial_guess]

    cons = {'type': 'ineq', 'fun': lambda x: constraint(x, demands, epsilon)}

    result = minimize(objective, initial_guess, args=(demands, costs), bounds=bounds, constraints=cons)
    return result.x, result.fun

# Parameters
mu = [100, 100, 100]
Sigma = [[2000, 1500, 500], [1500, 2000, 750], [500, 750, 2000]]
costs = [1, 1, 1]
epsilon = 0.05
budget = 10000

# Solve optimization
optimal_x, optimal_cost = solve_optimization(mu, Sigma, costs, epsilon, budget)

print(f"Optimal capacities: {optimal_x}")
print(f"Optimal cost: {optimal_cost}")

Optimal capacities: [149.66868688 194.77969171 256.70530331]
Optimal cost: 601.1536819010591


## o1

In [10]:
completion = openai_client.chat.completions.create(
    model="o1",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [11]:
print(response)

Solution Explanation

Below I outline two main solution approaches:

----------------------------------------------------------------------------
1. Analytical / Closed-Form Discussion
----------------------------------------------------------------------------

Because demands (X₁, X₂, X₃) follow a trivariate Normal(μ, Σ), we want:

  P_stockout(x) = P(X₁ > x₁ ∪ X₂ > x₂ ∪ X₃ > x₃) ≤ ε.

Equivalently,

  1 - P(X₁ ≤ x₁, X₂ ≤ x₂, X₃ ≤ x₃) ≤ ε  ⇒  P(X₁ ≤ x₁, X₂ ≤ x₂, X₃ ≤ x₃) ≥ 1 - ε.

In principle, this is

  Φ₃(x₁, x₂, x₃; μ, Σ) ≥ 1 - ε,

where Φ₃ is the trivariate Normal cumulative distribution function. However, for m=3 with general covariance, there is no simple closed-form expression for Φ₃. One could use numerical integration routines (e.g., “mvncdf” in some software libraries), but in many real settings the dimension m can be bigger, or one may simply lack a ready numerical integrator. That motivates a sampling-based approach.

-----------------------------------------------------

In [25]:
import numpy as np

def estimate_stockout_probability(x, demands):
    """
    Given a capacity vector x = (x1, x2, x3) and an array of demands
    of shape (N,3), returns the fraction of times at least one facility
    stocks out.
    """
    # stockout occurs if any demand Xi_j > x_j
    stockouts = np.any(demands > x, axis=1)  # boolean array
    return np.mean(stockouts)

def simulation_approach(
    mu=[100, 100, 100],
    Sigma=[[2000, 1500,  500],
           [1500, 2000,  750],
           [ 500,  750, 2000]],
    cost_coeff=[1, 1, 1],
    epsilon=0.05,
    budget=10000,
    grid_range=(120, 220),
    grid_step=5,
    N_sim=5000,
    random_seed=42
):
    """
    Performs a simple grid search over capacities in [grid_range[0], grid_range[1]] 
    with steps of grid_step. Uses N_sim samples for each candidate x to estimate 
    P_stockout(x), and returns a feasible solution with minimal cost. 
    """
    np.random.seed(random_seed)
    
    # Pre-generate demands once (common random numbers approach):
    # Large enough to keep the total number of calls <= budget.
    # For each capacity candidate, we'll reuse these draws.
    demands = np.random.multivariate_normal(mean=mu, cov=Sigma, size=N_sim)
    
    best_x = None
    best_cost = float('inf')
    
    # Build the discrete grid for each dimension
    x_values = np.arange(grid_range[0], grid_range[1] + grid_step, grid_step)

    for x1 in x_values:
        for x2 in x_values:
            for x3 in x_values:
                x_candidate = np.array([x1, x2, x3])
                
                # Estimate P(stockout)
                p_stockout_est = estimate_stockout_probability(x_candidate, demands)
                
                if p_stockout_est <= epsilon:
                    # Feasible, check cost
                    cost_candidate = cost_coeff[0]*x1 + cost_coeff[1]*x2 + cost_coeff[2]*x3
                    if cost_candidate < best_cost:
                        best_cost = cost_candidate
                        best_x = x_candidate.copy()
    
    return best_x, best_cost

if __name__ == "__main__":
    x_sol, cost_sol = simulation_approach()
    print("Best capacity vector:", x_sol)
    print("Estimated minimal cost subject to 5% stockout:", cost_sol)
    

Best capacity vector: [190 195 195]
Estimated minimal cost subject to 5% stockout: 580


## o3-mini

In [13]:
completion = openai_client.chat.completions.create(
    model="o3-mini",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [14]:
print(response)

Below is one complete way to “solve” this problem. In our write‐up we explain two possible approaches and then detail a simulation–based solution that uses a “parametric” search over capacity settings. (In our case the cost coefficients are equal so a “balanced” solution – setting each facility’s capacity equal to its own mean plus a safety margin measured in standard deviations – is a good candidate.) 

────────────────────────────
1. OVERVIEW OF THE APPROACH
────────────────────────────
A. Analytical (approximate) idea using quantiles

For each facility i the demand is N(μi, σii). One might be tempted to “set”
  P(Xi > x_i) = 1 – Φ((x_i – μi)/√(σii)) ≡ αi.
If the three facilities were independent, then
  P(no stockout) = ∏_{i=1}^3 (1 − αi)
so to enforce
  P(stockout) = 1 − ∏_{i=1}^3 (1 − αi) ≤ ε,
one simple (albeit conservative) idea is to set in a balanced way
  αi = 1 − (1 − ε)^(1/3).
For ε = 0.05, note that (1 – 0.05)^(1/3) ≈ 0.983 so αi ≃ 0.017.
Thus one would choose
  x_i = μ_i 

In [15]:
import numpy as np
from scipy.stats import norm, multivariate_normal

# ---------------------------
# Parameters and Data Setup:
# ---------------------------
m = 3
mu = np.array([100, 100, 100])
# Covariance matrix
Sigma = np.array([[2000, 1500, 500],
                  [1500, 2000, 750],
                  [500,  750, 2000]])
# Unit cost coefficients (assumed equal here)
c = np.ones(m)
# Risk tolerance
epsilon = 0.05
# Number of simulation replications per evaluation:
# You might want to adjust this so that the total number of evaluations is under BUDGET.
N_sim = 5000    # per evaluation (you can adjust this based on BUDGET)
np.random.seed(1234)  # for reproducibility

# Precompute the standard deviations for each facility
sigma_diag = np.sqrt(np.diag(Sigma))

# ---------------------------
# Simulation function:
# ---------------------------
def estimate_stockout_probability(x, N=N_sim):
    """
    Given capacity vector x, simulate N samples from the multivariate normal distribution
    with mean mu and covariance Sigma, and return the estimated probability of (at least one)
    stockout (i.e. any component demand > corresponding capacity).
    Also returns confidence interval half-width (approx) using normal approximation.
    """
    # Generate an (N x m) array of demands.
    demands = np.random.multivariate_normal(mu, Sigma, size=N)
    # stockout_flag: a boolean array saying if at least one component exceeds corresponding x
    stockout_flags = np.any(demands > x, axis=1)
    p_est = np.mean(stockout_flags)
    # Compute approximate standard error for a proportion:
    se = np.sqrt(p_est*(1-p_est)/N)
    return p_est, se


# ---------------------------
# Binary Search on z:
# ---------------------------
# We parameterize x = mu + z*sigma_diag (componentwise). 
# Our aim is to find the smallest z >= 0 such that estimated P_stockout(x) <= epsilon.

z_low = 0.0
z_high = 6.0   # a high enough number so that stockout-probability is very low
tol = 0.01     # tolerance on z
max_iters = 20

final_z = None
final_prob = None

for it in range(max_iters):
    z_mid = (z_low + z_high)/2
    x_candidate = mu + z_mid * sigma_diag
    p_stockout, se = estimate_stockout_probability(x_candidate)
    
    print(f"Iteration {it+1}: z = {z_mid:.3f}, candidate capacity = {x_candidate}, "
          f"Estimated P_stockout = {p_stockout:.4f} (SE ~ {se:.4f})")
    
    if p_stockout <= epsilon:
        # Candidate satisfies constraint, so try to lower z
        final_z = z_mid
        final_prob = p_stockout
        z_high = z_mid   # tighten the upper bound
    else:
        # Too many stockouts, increase z
        z_low = z_mid
        
    if (z_high - z_low) < tol:
        break

# Final candidate x:
final_x = mu + final_z * sigma_diag
total_cost = np.sum(c * final_x)

print("\n==================================================")
print("Final solution:")
print(f"Safety margin parameter z = {final_z:.3f}")
print(f"Capacity vector x = {final_x}")
print(f"Total installation cost = {total_cost:.2f}")
print(f"Final estimated stockout probability = {final_prob:.4f}")
print("==================================================")

Iteration 1: z = 3.000, candidate capacity = [234.16407865 234.16407865 234.16407865], Estimated P_stockout = 0.0034 (SE ~ 0.0008)
Iteration 2: z = 1.500, candidate capacity = [167.08203932 167.08203932 167.08203932], Estimated P_stockout = 0.1512 (SE ~ 0.0051)
Iteration 3: z = 2.250, candidate capacity = [200.62305899 200.62305899 200.62305899], Estimated P_stockout = 0.0340 (SE ~ 0.0026)
Iteration 4: z = 1.875, candidate capacity = [183.85254916 183.85254916 183.85254916], Estimated P_stockout = 0.0776 (SE ~ 0.0038)
Iteration 5: z = 2.062, candidate capacity = [192.23780407 192.23780407 192.23780407], Estimated P_stockout = 0.0478 (SE ~ 0.0030)
Iteration 6: z = 1.969, candidate capacity = [188.04517661 188.04517661 188.04517661], Estimated P_stockout = 0.0624 (SE ~ 0.0034)
Iteration 7: z = 2.016, candidate capacity = [190.14149034 190.14149034 190.14149034], Estimated P_stockout = 0.0546 (SE ~ 0.0032)
Iteration 8: z = 2.039, candidate capacity = [191.18964721 191.18964721 191.1896472

## Claude 3.5 Sonnet

In [19]:
response = claude_client.messages.create(
    model="claude-3-5-sonnet-20241022",  # Or another Claude 3 model
    max_tokens=4096,
    system="You are an expert in probability theory and stochastic modeling.",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.content[0].text)

I'll analyze this problem and propose a solution approach.

ANALYSIS:
1. The problem involves a joint chance constraint P(Xi_i > x_i for any i) ≤ epsilon.
2. Given multivariate normal demands, we can exploit properties of normal distributions.
3. The correlation structure makes this challenging, as we can't treat facilities independently.

SOLUTION APPROACH:
I'll propose a hybrid approach combining analytical insights with simulation:

1. For the analytical part:
- The stockout probability involves the complement of the CDF of a multivariate normal.
- We can use the fact that individual demands are normally distributed.

2. For optimization:
- Use gradient-free optimization with simulation to evaluate probabilities.
- Implement a binary search-like approach for each dimension.

Here's the implementation:

```python
import numpy as np
from scipy.stats import multivariate_normal
from typing import Tuple, List

class StockoutOptimizer:
    def __init__(self, 
                 m: int,
    

In [20]:
import numpy as np
from scipy.stats import multivariate_normal
from typing import Tuple, List

class StockoutOptimizer:
    def __init__(self, 
                 m: int,
                 mu: np.ndarray,
                 Sigma: np.ndarray,
                 costs: np.ndarray,
                 epsilon: float,
                 budget: int = 10000):
        self.m = m
        self.mu = mu
        self.Sigma = Sigma
        self.costs = costs
        self.epsilon = epsilon
        self.budget = budget
        self.mvn = multivariate_normal(mu, Sigma)
        
    def simulate_stockout_prob(self, x: np.ndarray, n_samples: int = 1000) -> Tuple[float, float]:
        """Simulate stockout probability for given capacity vector"""
        samples = self.mvn.rvs(size=n_samples)
        stockouts = np.any(samples > x, axis=1)
        p_stockout = np.mean(stockouts)
        std_error = np.sqrt(p_stockout * (1 - p_stockout) / n_samples)
        return p_stockout, std_error
    
    def optimize(self) -> Tuple[np.ndarray, float]:
        """Find optimal capacities using binary search with simulation"""
        # Initial bounds based on mean and standard deviation
        std_devs = np.sqrt(np.diag(self.Sigma))
        lower = self.mu
        upper = self.mu + 4 * std_devs  # Start with μ + 4σ as upper bound
        
        best_x = None
        best_cost = float('inf')
        n_iterations = self.budget // 1000  # Use budget for number of iterations
        
        for _ in range(n_iterations):
            # Try middle point
            x = (lower + upper) / 2
            p_stockout, _ = self.simulate_stockout_prob(x)
            
            if p_stockout <= self.epsilon:
                # Solution is feasible, try to reduce capacity
                upper = x
                if np.sum(self.costs * x) < best_cost:
                    best_x = x.copy()
                    best_cost = np.sum(self.costs * x)
            else:
                # Solution is infeasible, increase capacity
                lower = x
        
        return best_x, best_cost

    def validate_solution(self, x: np.ndarray, n_validation: int = 5000) -> dict:
        """Validate solution with larger number of samples"""
        p_stockout, std_error = self.simulate_stockout_prob(x, n_validation)
        return {
            'capacity': x,
            'cost': np.sum(self.costs * x),
            'p_stockout': p_stockout,
            'std_error': std_error,
            'confidence_interval': (p_stockout - 1.96*std_error, 
                                  p_stockout + 1.96*std_error)
        }

# Example usage
if __name__ == "__main__":
    m = 3
    mu = np.array([100, 100, 100])
    Sigma = np.array([[2000, 1500, 500],
                     [1500, 2000, 750],
                     [500, 750, 2000]])
    costs = np.array([1, 1, 1])
    epsilon = 0.05
    
    optimizer = StockoutOptimizer(m, mu, Sigma, costs, epsilon)
    x_opt, cost = optimizer.optimize()
    
    # Validate solution
    validation = optimizer.validate_solution(x_opt)
    print("\nOptimal Solution:")
    print(f"Capacities: {x_opt}")
    print(f"Total Cost: {cost:.2f}")
    print(f"Estimated stockout probability: {validation['p_stockout']:.4f}")
    print(f"95% CI: {validation['confidence_interval']}")


Optimal Solution:
Capacities: [192.23780407 192.23780407 192.23780407]
Total Cost: 576.71
Estimated stockout probability: 0.0424
95% CI: (np.float64(0.03681470564456984), np.float64(0.04798529435543016))


## DeepSeek-R1

In [21]:
import numpy as np
from scipy.stats import norm

# Problem parameters
m = 3
mu = np.array([100, 100, 100])
cov = np.array([[2000, 1500, 500],
               [1500, 2000, 750],
               [500, 750, 2000]])
epsilon = 0.05
budget = 10000
np.random.seed(0)  # Seed for reproducibility

# Generate demand samples
samples = np.random.multivariate_normal(mu, cov, size=budget)

# Bonferroni initial solution
sigma = np.sqrt(np.diag(cov))
epsilon_i = epsilon / m
z = norm.ppf(1 - epsilon_i)
x = mu + sigma * z

# Compute initial stockout count
stockout = np.any(samples > x, axis=1)
current_count = np.sum(stockout)
print(f"Initial stockout count: {current_count} (target ≤ {epsilon * budget})")
print(f"Initial capacities: {np.round(x, 2)}, Total cost: {np.sum(x):.2f}\n")

# Coordinate descent to minimize capacities
changed = True
iteration = 0
while changed:
    changed = False
    for i in range(m):
        # Get samples not currently in stockout
        not_stockout_mask = ~stockout
        X_i = samples[not_stockout_mask, i]
        if len(X_i) == 0:
            continue  # All samples are stockouts, cannot reduce
        
        # Calculate allowable new stockouts
        K = int(epsilon * budget) - current_count
        if K <= 0:
            continue
        
        # Find the x_i that introduces up to K new stockouts
        sorted_Xi = np.sort(X_i)
        idx = max(len(sorted_Xi) - K, 0)
        new_xi = sorted_Xi[idx]
        
        if new_xi < x[i]:
            # Update x[i] and check new stockout count
            new_stockout_i = (samples[:, i] > new_xi) & not_stockout_mask
            new_total = current_count + np.sum(new_stockout_i)
            
            if new_total <= epsilon * budget:
                x[i] = new_xi
                stockout |= new_stockout_i
                current_count = new_total
                changed = True
                print(f"Iteration {iteration}: Lower x_{i+1} to {new_xi:.2f}, stockouts={current_count}")
    iteration += 1

print("\nFinal capacities:", np.round(x, 2))
print("Total cost:", np.sum(x))
print(f"Estimated stockout probability: {current_count / budget:.4f} (≤ {epsilon})")

Initial stockout count: 415 (target ≤ 500.0)
Initial capacities: [195.17 195.17 195.17], Total cost: 585.51

Iteration 0: Lower x_1 to 185.15, stockouts=499
Iteration 0: Lower x_2 to 195.05, stockouts=499
Iteration 0: Lower x_3 to 195.06, stockouts=499

Final capacities: [185.15 195.05 195.06]
Total cost: 575.2576802695017
Estimated stockout probability: 0.0499 (≤ 0.05)
